# Knitting

Two types of stitches:
* knit stich (legs faced)
* purl stich (head faced)

Each cell contains 2 dependant knots: to legs of northener stich, head of southerner stich.

Any neighbour combinations are possible.


In [1]:
from random import random
import numpy as np
import sympy as sp
from ipywidgets import widgets
import k3d

from myutils import *
from mycurves import *

t, s, u, v, r, a, b, c, d = sp.symbols("t, s, u, v, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P0 P1 P2 P3')

In [2]:
from matplotlib.cm import get_cmap
palette = hexpalette(get_cmap('tab10'))

# Energy model

Energy model by [Choi and Lo, 2003]

* $t \in [0,1]$ for half loop, $t = 0.5$ in the middle of leg
* $w$ — loop width
* $c + 2e$ — total loop height with overlapping
* $c$ — distance between knots 
* $t_h$ — fabric thickness
* $d$ — yarn diameter
* $a$ — fuckery

$x(t) = a t^3 - 1.5 a t^2 + 0.5 (a + w) t$

$y(t) = 0.5(c + 2 e)(1 - cos (\pi t))$

$z(t) = 0.5(t_h - d)(1 - cos (2\pi t))$

* $\chi d$ — horizontal loop separation (distance in knot), $1 ≤ \chi ≤ \frac{w}{2d} - 1$
* $t = \beta_1, \beta_2 $ — righmost and leftmost points of knot $x(\beta_1)  - x(\beta_2) = \chi d$
* $t = \beta_3, \beta_4 $ — first intersection point $x(\beta_4) = x(\beta_3), y(\beta_4) - y(\beta_3) = c$

$\beta_{1,2} = \frac{1}{2}\left(1 ± \sqrt{\frac{a - 2w}{3a}}\right)$

$e = \frac{c}{2}\left(\frac{1}{cos \pi \beta_1} - 1\right)$

$t_h = d \left( \frac{2}{cos 2 \pi \beta_3 - cos 2 \pi \beta_4} + 1\right)$

In [3]:
Ka, Kw, Kc, Ke, Kz = sp.symbols("Ka, Kw, Kc, Ke, Kz")
Q = vector(
    Ka * t**3 - 1.5 * Ka * t**2 + 0.5*(Ka + Kw)*t,
    0.5 * (Kc + Ke) * (1 - sp.cos(sp.pi * t)),       # -e to make course width constant
    0.5 * Kz * (1 - sp.cos(2 * sp.pi * t)),
)

In [4]:
controls = {
    'Ka': widgets.FloatSlider(description="Ka", value=0.5, min=0, max=10.0),
    'Kw': widgets.FloatSlider(description="Kw", value=1.0, min=0, max=2.0),
    'Kc': widgets.FloatSlider(description="Kc", value=1.0, min=0, max=2.0),    
    'Ke': widgets.FloatSlider(description="Ke", value=0.5, min=0, max=2.0),
    'Kz': widgets.FloatSlider(description="Kz", value=0.25, min=0, max=2.0)
}
plot = k3d.plot()
widgets.GridBox([
    widgets.VBox(list(controls.values())),
    plot], 
    layout=widgets.Layout(grid_template_columns="auto auto"))

GridBox(children=(VBox(children=(FloatSlider(value=0.5, description='Ka', max=10.0), FloatSlider(value=1.0, de…

In [5]:
curve1 = k3d.line(vertices=[], color=palette[1])
plot += curve1
curve2 = k3d.line(vertices=[], color=palette[2])
plot += curve2

In [6]:
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    Q_ = Q.subs(params)    
    points = eval_curve(Q_, t, 36)    
    points1 = np.concatenate([points, np.flip(points,0) * np.array(vector(-1, 1, 1))])
    points2 = points1 + np.array(vector(0, params['Kc'], 0))
    curve1.vertices = points1.astype(np.float32)
    curve2.vertices = points2.astype(np.float32)

In [7]:
redraw()

In [8]:
for w in controls.values():
    w.observe(redraw, 'value')

# Spline model

Quadratic uniform spline.

In [3]:
Ka, Kw, Kc, Ke, Kz = sp.symbols("Ka, Kw, Kc, Ke, Kz")
cpoints = [
    vector(-1.0-Ka,-Kc,-Kz),
    vector(-1.0+Ka,-Kc,-Kz),
    vector(-1.0+Kw,-Ke,+Kz),
    vector( 0.0-Kw, Ke,+Kz),
    vector( 0.0-Ka, Kc,-Kz),
    vector( 0.0+Ka, Kc,-Kz),
]
splines = [Spline(u, B2, cpoints[ids]) for ids in iter_slices(3, len(cpoints))]

In [20]:
controls = {
    'Ka': widgets.FloatSlider(description="Ka", value=1/3, min=0, max=1.0, continuous_update=False),
    'Kw': widgets.FloatSlider(description="Kw", value=2/3, min=0, max=1.0, continuous_update=False),
    'Kc': widgets.FloatSlider(description="Kc", value=0.75, min=0, max=1.0, continuous_update=False),    
    'Ke': widgets.FloatSlider(description="Ke", value=0.25, min=0, max=1.0, continuous_update=False),
    'Kz': widgets.FloatSlider(description="Kz", value=0.25, min=0, max=1.0, continuous_update=False)
}
plot = k3d.plot()
widgets.GridBox([widgets.VBox(list(controls.values())), plot], layout=widgets.Layout(grid_template_columns="auto auto"))

GridBox(children=(VBox(children=(FloatSlider(value=0.3333333333333333, continuous_update=False, description='K…

In [21]:
plot += k3d.points(positions=[], color=palette[0], point_size=0.05)
plot += k3d.line(vertices=[], color=palette[1])
plot += k3d.line(vertices=[], color=palette[2])

In [22]:
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    points0 = np.array([p.subs(params) for p in cpoints])
    points = np.concatenate([eval_curve(spl.subs(params), u) for spl in splines])
    points1 = np.concatenate([points, np.flip(points,0) * np.array(vector(-1, 1, 1))])
    points2 = points1 + np.array(vector(0, 1.0, 0))    
    plot.objects[0].positions = points0.astype(np.float32)
    plot.objects[1].vertices = points1.astype(np.float32)
    plot.objects[2].vertices = points2.astype(np.float32)
    
    

In [23]:
redraw()

In [24]:
for w in controls.values():
    w.observe(redraw, 'value')

# Topology - hexagonal

From ["3D Modeling of Plain Weft Knitted Structures of Compressible Yarn", Kyosev et al, 2010] 

* $A^{NW}, A^{NE}$ — intersection points near the tip of a loop head
* $A^{SW}, A^{SE}$ — intersection points at sides of a loop head

Symmetrical variations $z+$ and $z-$ for above/below yarn.

* Knit stitch:
  * $A^{NW}_z = A^{NE}_z = -r$
  * $A^{SW}_z = A^{SE}_z = +r$
* Purl stitch:
  * $A^{NW}_z = A^{NE}_z = +r$
  * $A^{SW}_z = A^{SE}_z = -r$

In [3]:
base = np.array([
    [1, 0, -1],
    [2, 1, +1],
    [1, 2, +1],
    [2, 3, -1],
    [4, 3, -1],
    [5, 2, +1],
    [4, 1, +1],
    [5, 0, -1]    
]) * np.array([1/6, 1/2, 1]) + np.array([0.0, 0.25, 0.0])

def plain():
    stitch = base
    row1 = np.concatenate([stitch + np.array([0, 0, 0]), stitch + np.array([1, 0, 0]), stitch + np.array([2, 0, 0])])
    row2 = np.concatenate([stitch + np.array([0, 1, 0]), stitch + np.array([1, 1, 0]), stitch + np.array([2, 1, 0])])
    row3 = np.concatenate([stitch + np.array([0, 2, 0]), stitch + np.array([1, 2, 0]), stitch + np.array([2, 2, 0])])
    return row1, row2, row3

def rib():
    knit = base 
    purl = base * np.array([1, 1, -1]) + np.array([0, 0, -2])
    row1 = np.concatenate([knit, purl + np.array([6, 0, 0]), knit + np.array([12, 0, 0])])
    row2 = row1 + np.array([0, 2, 0])
    row3 = row1 + np.array([0, 4, 0])
    return row1, row2, row3

def garter():
    headrisen = np.copy(base) 
    headrisen[3] = headrisen[3] * np.array([1, 1, -1]) + np.array([0, 0, 2])
    headrisen[4] = headrisen[4] * np.array([1, 1, -1]) + np.array([0, 0, 2])
    purl = base * np.array([1, 1, -1]) + np.array([0, 0, 2])
    backrisen = np.copy(base)
    backrisen[0] = backrisen[0] * np.array([1, 1, -1]) + np.array([0, 0, 2])
    backrisen[-1] = backrisen[-1] * np.array([1, 1, -1]) + np.array([0, 0, 2])
    row1 = np.concatenate([headrisen, headrisen + np.array([6, 0, 0]), headrisen + np.array([12, 0, 0])])
    row2 = np.concatenate([purl + np.array([0, 2, 0]), purl + np.array([6, 2, 0]), purl + np.array([12, 2, 0])])
    row3 = np.concatenate([backrisen + np.array([0, 4, 0]), backrisen + np.array([6, 4, 0]), backrisen + np.array([12, 4, 0])])
    return row1, row2, row3

def moss():
    knit = base
    purl = base * np.array([1, 1, -1])
    row1 = np.concatenate([knit, purl + np.array([6, 0, 0]), knit + np.array([12, 0, 0])])
    row2 = np.concatenate([purl + np.array([0, 2, 0]), knit + np.array([6, 2, 0]), purl + np.array([12, 2, 0])])
    row3 = np.concatenate([knit + np.array([0, 4, 0]), purl + np.array([6, 4, 0]), knit + np.array([12, 4, 0])])
    return row1, row2, row3
    

In [4]:
controls = {
    'kind': widgets.RadioButtons(
        value='plain',
        options=['plain', 'rib', 'garter', 'moss'],
        description='Kind:')}
plot = k3d.plot()
widgets.GridBox([widgets.VBox(list(controls.values())), plot], layout=widgets.Layout(grid_template_columns="auto auto"))

GridBox(children=(VBox(children=(RadioButtons(description='Kind:', options=('plain', 'rib', 'garter', 'moss'),…

In [5]:
plot += k3d.vectors(np.array([[0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0]]), np.array([[0, 4, 0], [0, 4, 0], [0, 4, 0], [0, 4, 0]]), color=0x000000, use_head=False)
plot += k3d.vectors(np.array([[0.5, 0, 0], [1.5, 0, 0], [2.5, 0, 0]]), np.array([[0, 4, 0], [0, 4, 0], [0, 4, 0]]), color=0x808080, use_head=False)
plot += k3d.vectors(np.array([[0, 0, 0], [0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0]]), np.array([[3, 0, 0], [3, 0, 0], [3, 0, 0], [3, 0, 0], [3, 0, 0]]), color=0x000000, use_head=False)

/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [6]:
row1points = k3d.points(positions=[], point_size=0.05, color=palette[1])
row1line = k3d.line(vertices=[], color=palette[1])
row2points = k3d.points(positions=[], point_size=0.05, color=palette[2])
row2line = k3d.line(vertices=[], color=palette[2])
row3points = k3d.points(positions=[], point_size=0.05, color=palette[3])
row3line = k3d.line(vertices=[], color=palette[3])
plot += row1line
plot += row2line
plot += row3line
plot += row1points
plot += row2points
plot += row3points

In [17]:
def redraw():
    params = {k: w.value for k, w in controls.items()}
    kind = params['kind'] 
    if kind == 'plain':
        rows = plain()
    elif kind == 'rib':
        rows = rib()
    elif kind == 'garter':
        rows = garter()
    elif kind == 'moss':
        rows = moss()
    else:
        return
    rows = [r * np.array([1, 1, 0.25]) for r in rows]
    
    row1points.positions = rows[0].astype(np.float32)
    row1line.vertices = rows[0].astype(np.float32)    
    row2points.positions = rows[1].astype(np.float32)
    row2line.vertices = rows[1].astype(np.float32)    
    row2spline.vertices = eval_spline(B2, u, rows[1].astype(np.float32))
    row3points.positions = rows[2].astype(np.float32)
    row3line.vertices = rows[2].astype(np.float32)    

In [18]:
redraw()

In [9]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')

## Adjusting Topology

Main hypotesis: shape of a loop (2 knots) only depends on types of N, S, W, E neighbours

Constraint: z-distance in anchors should remain 2 — failing because of "flattening"

In [90]:
base = {
    'hnw': vector(-1/6, 3/4, -1),
    'hsw': vector(-2/6, 1/4, +1),
    'lnw': vector(-1/6,-1/4, +1),
    'lsw': vector(-2/6,-3/4, -1),
    'hne': vector(+1/6, 3/4, -1),
    'hse': vector(+2/6, 1/4, +1),
    'lne': vector(+1/6,-1/4, +1),
    'lse': vector(+2/6,-3/4, -1),
}

def stitch(val):
    return {k: vector(v[0], v[1], v[2] * val) for k, v in base.items()} 

In [99]:
def topology(pattern):
    stitches = [[stitch(v) for v in row] for row in pattern]
    Km = pattern[1][1]
    Kw = pattern[1][0]
    Ke = pattern[1][2]
    Kn = pattern[0][1]
    Ks = pattern[2][1]
    
    Ksw = pattern[2][0]
    Kse = pattern[2][2]
    Knw = pattern[0][0]
    Kne = pattern[0][2]
    
    Sm = stitches[1][1]
    Sw = stitches[1][0]
    Se = stitches[1][2]
    Sn = stitches[0][1]
    Ss = stitches[2][1]
    
    Ssw = stitches[2][0]
    Sse = stitches[2][2]
    Snw = stitches[0][0]
    Sne = stitches[0][2]

    def zshift(stitch, keys, d):
        for k in keys:
            stitch[k][2] += d

    def fix_w(S, K, Kw):
        if K != Kw:
            zshift(S, ('lsw',), K)
            zshift(S, ('hsw', 'hnw', 'lnw'), 0.5 * K)
            
    def fix_e(S, K, Ke):
        if K != Ke:
            zshift(S, ('lse',), K)
            zshift(S, ('hse', 'hne', 'lne'), 0.5 * K)

    fix_w(Sm, Km, Kw)
    fix_w(Sn, Kn, Knw)
    fix_w(Ss, Ks, Ksw)

    fix_w(Se, Ke, Km)
    fix_w(Sne, Kne, Kn)
    fix_w(Sse, Kse, Ks)

    fix_e(Sm, Km, Ke)
    fix_e(Sn, Kn, Kne)
    fix_e(Ss, Ks, Kse)
    
    fix_e(Sw, Kw, Km)
    fix_e(Snw, Knw, Kn)
    fix_e(Ssw, Ksw, Ks)

    def fix_s(S, K, Ks):
        if K != Ks:
            zshift(S, ('lsw', 'lse'), 2 * K)
            zshift(S, ('lnw', 'lne'), -2 * K)

    def fix_n(S, K, Kn):
        if K != Kn:
            zshift(S, ('hsw', 'hse'), -0.5 * K)
            
    fix_s(Sm, Km, Ks)        
    fix_s(Sw, Kw, Ksw)        
    fix_s(Se, Ke, Kse)        

    fix_s(Sn, Kn, Km)        
    fix_s(Snw, Knw, Kw)        
    fix_s(Sne, Kne, Ke)
    
    fix_n(Sm, Km, Kn)
    fix_n(Sw, Kw, Knw)
    fix_n(Se, Ke, Kne)

    fix_n(Ss, Ks, Km)        
    fix_n(Ssw, Ksw, Kw)        
    fix_n(Sse, Kse, Ke)
    
    return stitches

In [81]:
plot = k3d.plot()
plot += k3d.vectors(np.array([[0.01, 0, 0], [1, 0, 0], [2, 0, 0], [2.99, 0, 0]]), 
                    np.array([[0, 4, 0], [0, 4, 0], [0, 4, 0], [0, 4, 0]]), color=0x000000, use_head=False)
plot += k3d.vectors(np.array([[0.5, 0, 0], [1.5, 0, 0], [2.5, 0, 0]]), 
                    np.array([[0, 4, 0], [0, 4, 0], [0, 4, 0]]), color=0x808080, use_head=False)
plot += k3d.vectors(np.array([[0.01, 0, 0], [0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0]]), 
                    np.array([[2.99, 0, 0], [2.99, 0, 0], [2.99, 0, 0], [2.99, 0, 0], [2.99, 0, 0]]), color=0x000000, use_head=False)

/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [82]:
row1points = k3d.points(positions=[], point_size=0.05, color=palette[1], shader='3d', opacity=0.5)
row1line = k3d.line(vertices=[], color=palette[1])
row2points = k3d.points(positions=[], point_size=0.05, color=palette[2], shader='3d', opacity=0.5)
row2line = k3d.line(vertices=[], color=palette[2])
row3points = k3d.points(positions=[], point_size=0.05, color=palette[3], shader='3d', opacity=0.5)
row3line = k3d.line(vertices=[], color=palette[3])
plot += row1line
plot += row2line
plot += row3line
plot += row1points
plot += row2points
plot += row3points

In [83]:
checkbox = lambda:widgets.Checkbox(value=True, indent=False) 
skip = lambda:widgets.Checkbox(value=False, disabled=True, indent=False)
pattern = widgets.VBox([
    widgets.HBox([checkbox(), checkbox(), checkbox()]),                  # NW NC NE
    widgets.HBox([checkbox(), checkbox(), checkbox()]),          # CW CC CE
    widgets.HBox([checkbox(), checkbox(), checkbox()]),                  # SW SC SE
])
def read_pattern():
    return [[int(w.value)*2-1 if not w.disabled else 0 for w in row.children] for row in pattern.children]

In [84]:
widgets.GridBox([pattern, plot], layout=widgets.Layout(grid_template_columns="100px auto"))

GridBox(children=(VBox(children=(HBox(children=(Checkbox(value=True, indent=False), Checkbox(value=True, inden…

In [92]:
def ancpoints(stitch):
    return np.array([
        stitch['lsw'], 
        stitch['lnw'], 
        stitch['hsw'], 
        stitch['hnw'],
        stitch['hne'], 
        stitch['hse'], 
        stitch['lne'], 
        stitch['lse']
    ], dtype=np.float32).reshape(-1, 3)

Ks = vector(1/12, 0, 0)

def splpoints(stitch):
    return np.array([
        stitch['lsw']+Ks,
        stitch['lnw']+Ks, 
        stitch['hsw']-Ks, 
        stitch['hnw']-Ks,
        stitch['hne']+Ks, 
        stitch['hse']+Ks, 
        stitch['lne']-Ks, 
        stitch['lse']-Ks
    ], dtype=np.float32).reshape(-1, 3)

def rowpoints(stitches, func):    
    return np.concatenate([func(stitches[0]), 
                           func(stitches[1]) + np.array([1, 0, 0]), 
                           func(stitches[2]) + np.array([2, 0, 0])]) * np.array([1, 1, 0.25]) + np.array([0.5, 1, 0])

def redraw():
    stitches = topology(read_pattern())
    
    ancpoints1 = rowpoints(stitches[0], ancpoints) + np.array([0, 2, 0])
    ancpoints2 = rowpoints(stitches[1], ancpoints) + np.array([0, 1, 0])
    ancpoints3 = rowpoints(stitches[2], ancpoints)
    splpoints1 = rowpoints(stitches[0], splpoints) + np.array([0, 2, 0])
    splpoints2 = rowpoints(stitches[1], splpoints) + np.array([0, 1, 0])
    splpoints3 = rowpoints(stitches[2], splpoints)
    
    row1points.positions = ancpoints1
    row1line.vertices = eval_splines(B2,  splpoints1)
    row2points.positions = ancpoints2
    row2line.vertices = eval_splines(B2,  splpoints2)
    row3points.positions = ancpoints3
    row3line.vertices = eval_splines(B2,  splpoints3)

In [93]:
redraw()

In [94]:
for v in pattern.children:
    for w in v.children:
        w.observe(lambda *args: redraw(), 'value')

# Geometry

### Parameters
* real scale to get correct thickness
* yarn thickness

### Manual control
* wale width
* course width

### Constraints

* symmetry at cell boundaries to align spline segments
* distance between splines at knot point > r
* distance between splines and cell edges > r